In [19]:
from transformers import AutoTokenizer
from common import *
from experiments import *
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [2]:
import os
import random
import time
from contextlib import contextmanager
import numpy as np
import torch

@contextmanager
def simple_timer(message, logger=None):
    start_time = time.time()
    yield
    timer_message = f'{message}: {time.time() - start_time:.3f}  [s]'
    if logger is None:
        print(timer_message)
    else:
        logger.info(timer_message)
        
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

def compute_weights(y, target_positive_ratio):
    positive_ratio = y.mean()
    positive_weight = target_positive_ratio / positive_ratio
    negative_weight = (1 - target_positive_ratio) / (1 - positive_ratio)
    weights = np.full(len(y), negative_weight)
    weights[y == 1] = positive_weight
    return weights

In [3]:
import datasets
import pandas as pd

df = pd.read_csv('~/src/kaggle/train.csv',na_filter=False)
df['weight'] = compute_weights(df.is_duplicate, 0.174)
raw_dataset = datasets.Dataset.from_pandas(df)

In [4]:
raw_dataset.column_names

['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate', 'weight']

In [5]:
raw_dataset.shape

(404290, 7)

In [6]:
type(raw_dataset)

datasets.arrow_dataset.Dataset

In [7]:
raw_dataset[0]

{'id': 0,
 'qid1': 1,
 'qid2': 2,
 'question1': 'What is the step by step guide to invest in share market in india?',
 'question2': 'What is the step by step guide to invest in share market?',
 'is_duplicate': 0,
 'weight': 1.3094438628066831}

In [8]:
def preprocess(examples):
    return tokenizer(
        examples['question1'],
        examples['question2'],
        truncation='longest_first',
        max_length=max_length,
    )
max_length=83
remove_columns = ['id','qid1','qid2','question1','question2']    

In [9]:
dataset = raw_dataset.map(
    preprocess, batched=True, remove_columns=remove_columns
    )

Map:   0%|          | 0/404290 [00:00<?, ? examples/s]

In [10]:
dataset[0]

{'is_duplicate': 0,
 'weight': 1.3094438628066831,
 'input_ids': [101,
  2054,
  2003,
  1996,
  3357,
  2011,
  3357,
  5009,
  2000,
  15697,
  1999,
  3745,
  3006,
  1999,
  2634,
  1029,
  102,
  2054,
  2003,
  1996,
  3357,
  2011,
  3357,
  5009,
  2000,
  15697,
  1999,
  3745,
  3006,
  1029,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

トークナイザは、通常、特別なトークン（BERTベースのモデルにおける[SEP]のような）でそれらを分離しながら、これら2つのテキストを内部的に連結します。この分離トークンは、1つの質問がどこで終わり、もう1つの質問がどこで始まるかをモデルが理解するのに役立つため、非常に重要です。
トークン化：連結された文字列は、トークンのシーケンスに変換されます。トークン化は、トークナイザーの設定に応じて、テキストを単語、サブワード、または文字に分割します。
トークンIDに変換します：各トークンは、トークナイザーの語彙から一意の整数IDにマッピングされます。これが input_id と呼ばれるものです。
切り捨て：トークンの総数がmax_lengthを超えた場合、指定された戦略（この場合はlongest_first）に基づいてシーケンスが切り捨てられます。
パディング：シーケンスがmax_lengthより短い場合、指定された最大長に達するまで特別なパディングトークン（[PAD]など）でパディングされます。

In [11]:
from transformers import DataCollatorWithPadding

collater_fn = DataCollatorWithPadding(tokenizer=tokenizer,padding='longest')


2024-05-01 02:45:11.314402: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 02:45:11.314476: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 02:45:11.315773: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-01 02:45:12.689882: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [25]:
from torch.utils.data import DataLoader

data_loader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    drop_last=True,
    num_workers=2,
    collate_fn = collater_fn
)

In [26]:
class TransformerModelV1(nn.Module):
    def __init__(self, model_name_or_path, dropout=0.1):
        super(TransformerModelV1, self).__init__()
        self.encoder = AutoModel.from_pretrained(mdoel_name_or_path)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(self.encoder.config.hidden_size, 1)
        
    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        outputs = self.encoder(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
        )
        return self.linear(self.dropout(outputs[1]))

In [27]:
def setup_adamw_optimizer(model, learning_rate, weight_decay):
    no_decay = ['bias',"LayNorm.weight"]
    optimizer_parameters = [
        {
            "params":[
                p
                for n, p in model.named_parameters()
                if not any(nd in n for nd in no_decay)
            ],
            'weight_decay':weight_decay,
        },
        {
            "params":[
                p
                for n, p in model.named_parameters()
                if any(nd in n for nd in no_decay)
            ],
        }
    ]
    return AdamW(optimizer_parameters, lr=learning_rate)

In [28]:
from transformers import get_linear_schedule_with_warmup
#訓練セット（trn_dataset), エポック数(num_epochs), バッチサイズ（batch_size)から、
#学習全体のステップ数を求める
num_training_steps = len(trn_dataset) * num_epochs//batch_size


#学習全体のステップ数とウォームアップあっぷ期間の全学習機関に対する割合（warmup_step_ratio)から、
# ウォームアップ期間の学習ステップ数を求める
# warmup_step_ratioは0以上0.2以下の値が指定されることが多い
num_warmup_steps = int(num_training_steps * warmup_step_ratio)

#最適化アルゴリズムと学習ステップ数に関する情報を元に学習率スケジュールを作成する
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_training_steps, num_warmup_steps
)

NameError: name 'trn_dataset' is not defined